In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Materials Cloud

In [ ]:
import json
from os import path, getenv
from glob import glob
import ipywidgets as ipw
from pprint import pformat
from markdown import markdown
from importlib import import_module
from IPython.display import clear_output, Markdown
import json

In [ ]:
from config import aiidalab_home, aiidalab_apps

In [ ]:
def render():
    clear_output()
    apps = load_apps()
    config = read_config()
    for name in apps:
        accordion = mk_accordion(name)
        accordion.selected_index = None if name in config['hidden'] else 0
        display(accordion)

In [ ]:
def mk_accordion(name):
    launcher = load_widget(name)
    launcher.layout = ipw.Layout(width="900px")
    btn_box = mk_buttons(name)
    box = ipw.HBox([launcher, btn_box])
    accordion = ipw.Accordion(children=[box])
    title = load_title(name)
    accordion.set_title(0, title)
    on_change = lambda c: record_showhide(name, accordion.selected_index==0)
    accordion.observe(on_change, names="selected_index")
    return accordion

In [ ]:
def load_title(name):
    try:
        fn = path.join(aiidalab_apps, name, 'metadata.json')
        metadata = json.load(open(fn))
        title = metadata['title']
    except:
        title = "%s (couldn't load title)"%name
    return title

In [ ]:
def load_apps():
    apps = [path.basename(fn) for fn in glob(path.join(aiidalab_apps, '*')) if path.isdir(fn)]
    config = read_config()
    order = config['order']
    apps.sort(key=lambda x: order.index(x) if x in order else -1)
    config['order'] = apps
    write_config(config)
    return apps

In [ ]:
def load_widget(name):
    if path.exists(path.join(aiidalab_apps, name, 'start.py')):
        return load_start_py(name)
    else:  # fall back
        return load_start_md(name)

In [ ]:
def load_start_py(name):
    try:
        mod = import_module('apps.%s.start' % name)
        appbase = "../" + name
        jupbase = "../../.."
        notebase = jupbase+"/notebooks/apps/"+name
        try:
            return mod.get_start_widget(appbase=appbase, jupbase=jupbase, notebase=notebase)
        except:
            return mod.get_start_widget(appbase=appbase, jupbase=jupbase)
    except Exception as e:
        return ipw.HTML("<pre>%s</pre>" % str(e))

In [ ]:
def load_start_md(name):
    fn = path.join(aiidalab_apps, name, 'start.md')
    try:
        
        md_src = open(fn).read()
        md_src = md_src.replace("](./", "](../%s/"%name)
        html = markdown(md_src)

        # open links in new window/tab
        html = html.replace('<a ', '<a target="_blank" ')

        # downsize headings
        html = html.replace("<h3", "<h4")
        return ipw.HTML(html)

    except Exception as e:
        return ipw.HTML("Could not load start.md")

In [ ]:
def mk_buttons(name):
    layout = ipw.Layout(width="40px")
    btn_up   = ipw.Button(description=u"▲", layout=layout)
    btn_up.on_click(lambda b: move_updown(name, -1))
    btn_down = ipw.Button(description=u"▼", layout=layout)
    btn_down.on_click(lambda b: move_updown(name, +1))
    btn_box = ipw.VBox([btn_up, btn_down])
    return(btn_box)

In [ ]:
def move_updown(name, delta):
    config = read_config()
    order = config['order']
    n = len(order)
    i = order.index(name)
    del(order[i])
    j = min(n-1, max(0, i + delta))
    order.insert(j, name)
    config['order'] = order
    write_config(config)
    render()

In [ ]:
def record_showhide(name, visible):
    config = read_config()
    hidden = set(config['hidden'])
    if visible:
        hidden.discard(name)
    else:
        hidden.add(name)
    config['hidden'] = list(hidden)
    write_config(config)

In [ ]:
CONFIG_FN = ".launcher.json"
def read_config():
    if path.exists(CONFIG_FN):
        return json.load(open(CONFIG_FN,'r'))
    else:
        return {'order':[], 'hidden':[]} #default config

def write_config(config):
    json.dump(config, open(CONFIG_FN,'w'), indent=2)

In [ ]:
#start the app
render()